In [16]:
%run file_paths_updt Copy

StatementMeta(, 795606b0-112b-4455-bb44-86d8ff83b501, 18, Finished, Available, Finished)

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import sys

StatementMeta(, 795606b0-112b-4455-bb44-86d8ff83b501, 19, Finished, Available, Finished)

In [18]:
emp_dim = spark.read.format('delta').load(Gold+'//'+'DimEmployee')
loc_dim = spark.read.format('delta').load(Gold+'//'+'DimLocation')
dept_dim = spark.read.format('delta').load(Gold+'//'+'DimDepartment')
df = spark.read.format('delta').load(Silver+'//'+'hr_data_silver')

StatementMeta(, 795606b0-112b-4455-bb44-86d8ff83b501, 20, Finished, Available, Finished)

In [ ]:
%run file_paths_updt Copy

from pyspark.sql.functions import *
from pyspark.sql.window import Window
import sys

emp_dim = spark.read.format('delta').load(Gold+'//'+'DimEmployee')
loc_dim = spark.read.format('delta').load(Gold+'//'+'DimLocation')
dept_dim = spark.read.format('delta').load(Gold+'//'+'DimDepartment')
df = spark.read.format('delta').load(Silver+'//'+'hr_data_silver')

def fact_Creation():
    try:
        df4 = df.join(loc_dim, on=['location_city', 'location_state'], how='inner')
        df5 = df4.join(dept_dim, on='department', how='inner')

# Transformations
        df6 = df5.withColumn('total_employees', count('employee_id').over(Window.partitionBy('Department_id')))\
                 .withColumn('age', floor(months_between(current_date(), col("birthdate")) / 12))\
                 .withColumn('avg_age', floor(avg('age').over(Window.partitionBy('Department_id'))))\
                 .withColumn("gender", when(col("gender") == "M", "Male")
                              .when(col("gender") == "FM", "Female")
                              .otherwise('Other'))

# Gender distribution
        gender_counts = df6.groupBy("department", "gender").count()

        gender_percentages = gender_counts.withColumn(
         "percentage",
        round((col("count") / sum("count").over(Window.partitionBy("department"))) * 100, 2)
        ).withColumn(
        "gender_percentage",
         concat_ws("", col("gender"), lit(":"), col("percentage").cast("string"), lit("%"))
        )

        df_7 = gender_percentages.groupBy("department").agg(
        concat_ws(", ", collect_list("gender_percentage")).alias("gender_distribution")
        )

        df_8 = df6.join(df_7, on='department', how='inner')

# Inactive employee calculation
        df_with_in_active = df_8.withColumn(
        'inactive_emp',
        sum(when(col('termdate').isNotNull(), 1).otherwise(0)).over(Window.partitionBy('Department_id'))
        )

# Final fact table
        window_spec = Window.orderBy('employee_id')

        hr_fact = df_with_in_active.withColumn(
        'turnover_rate', round((col('inactive_emp') / col('total_employees')) * 100, 2)
        ).withColumn(
        'fact_id', row_number().over(window_spec)
        ).select(
        'fact_id', 'emp_id_int', 'employee_id', 'location_id', 'department_id',
        'total_employees', 'avg_age', 'gender_distribution', 'turnover_rate'
        )

# Save to Gold path
        hr_fact.write.format('delta').mode('overwrite').save(Gold + '//' + 'Fact_Table')
        print('count', hr_fact.count())     
    except:
        print(sys.exc_info())

StatementMeta(, 795606b0-112b-4455-bb44-86d8ff83b501, 22, Finished, Available, Finished)

In [11]:
fact_Creation()

StatementMeta(, ff612dc8-62ad-41b5-ac5a-a57e16f89141, 12, Finished, Available, Finished)

count 22214
